In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
from huggingface_hub import login

login(...)

# New Section

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

Finetune model normally

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load small dataset (already 300 samples)
dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="train")

# Load tokenizer
model_id = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Load model with memory optimization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    low_cpu_mem_usage=True
)
model.config.use_cache = False

# Format dataset
def format_example(example):
    prompt = (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer: {example['answer']}"
    )
    return {"text": prompt}

formatted_dataset = dataset.map(format_example)

# Tokenize with truncation only (no padding)
MAX_LENGTH = 512
tokenized_dataset = formatted_dataset.map(
    lambda e: tokenizer(e["text"], truncation=True, max_length=MAX_LENGTH, padding=False),
    batched=True,
    remove_columns=dataset.column_names
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments optimized for A100
training_args = TrainingArguments(
    output_dir="./llama3-qa",
    logging_dir=None,
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    save_strategy="no",
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=False,          # Disable fp16
    bf16=True,           # Enable bf16 (A100 optimized)
    gradient_checkpointing=True,
    dataloader_pin_memory=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


Using device: cuda


PackageNotFoundError: No package metadata was found for bitsandbytes

In [ ]:
trainer.save_model("./llama3-qa")
tokenizer.save_pretrained("./llama3-qa")

('./llama3-qa/tokenizer_config.json',
 './llama3-qa/special_tokens_map.json',
 './llama3-qa/tokenizer.json')

Generate predictions from the finetuned model

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer from local path
model_dir = "/content/llama3-qa"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32)
model.eval()

# Load test split of dataset
test_dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="test")

# Format function
def format_prompt(example):
    return (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer:"
    )

# Inference
results = []
for example in tqdm(test_dataset, desc="Generating answers"):
    prompt = format_prompt(example)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.7,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_answer = generated.split("Answer:")[-1].strip()

    results.append({
        "question": example["question"],
        "context": example["text"],
        "true_answer": example["answer"],
        "predicted_answer": predicted_answer
    })

# Save to JSON
output_path = "predicted_answers.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Saved predictions to {output_path}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating answers:   0%|          | 0/51 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Generating answers: 100%|██████████| 51/51 [01:57<00:00,  2.30s/it]

Saved predictions to predicted_answers.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/llama3-qa/model-00001-of-00002.safetensors /content/drive/MyDrive/

In [ ]:
!cp /content/llama3-qa/model-00002-of-00002.safetensors /content/drive/MyDrive/

In [ ]:
from google.colab import files
files.download('/content/llama3-qa/model-00001-of-00002.safetensors')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Finetune model with LoRA

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load small dataset
dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="train")

# Load tokenizer
model_id = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Configure 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
peft_config = LoraConfig(
    r=8,                   # Rank
    lora_alpha=32,         # Scaling factor
    lora_dropout=0.05,     # Dropout
    target_modules=["q_proj", "v_proj"],  # Target modules
    task_type="CAUSAL_LM",
    bias="none"            # No bias for LoRA
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Show trainable parameters count

# Format dataset
def format_example(example):
    prompt = (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer: {example['answer']}"
    )
    return {"text": prompt}

formatted_dataset = dataset.map(format_example)

# Tokenize dataset
MAX_LENGTH = 512
tokenized_dataset = formatted_dataset.map(
    lambda e: tokenizer(
        e["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False
    ),
    batched=True,
    remove_columns=formatted_dataset.column_names
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Optimized training arguments
training_args = TrainingArguments(
    output_dir="./llama3-qa-lora",
    per_device_train_batch_size=2,          # Increased due to LoRA memory efficiency
    gradient_accumulation_steps=4,           # Adjust based on GPU memory
    num_train_epochs=3,
    learning_rate=1e-4,                     # Lower LR for adapter training
    weight_decay=0.01,
    bf16=True,                              # Use bfloat16
    logging_steps=10,
    optim="paged_adamw_8bit",               # Optimizer for 8-bit models
    save_strategy="no",
    report_to="none",
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training
trainer.train()

# Save adapter weights
model.save_pretrained("llama3-qa-lora-adapter")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


Map:   0%|          | 0/349 [00:00<?, ? examples/s]

<ipython-input-1-7de51740effe>:98: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.475000
20,1.935500
30,1.444400
40,1.168200
50,1.085800
60,1.080500
70,1.043600
80,1.047700
90,1.015400
100,1.005400


Generate predictions for the LORA-finetuned model

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import json
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load adapter configuration
adapter_dir = "/content/llama3-qa-lora-adapter"  # Path to saved LoRA adapter
config = PeftConfig.from_pretrained(adapter_dir)

# Load base model with quantization (must match training setup)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False
)

# Attach LoRA adapter to base model
model = PeftModel.from_pretrained(base_model, adapter_dir)
model.eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load test dataset
test_dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="test")

# Format function
def format_prompt(example):
    return (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer:"
    )

# Inference
results = []
for example in tqdm(test_dataset, desc="Generating answers"):
    prompt = format_prompt(example)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.7,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_answer = generated.split("Answer:")[-1].strip()

    results.append({
        "question": example["question"],
        "context": example["text"],
        "true_answer": example["answer"],
        "predicted_answer": predicted_answer
    })

# Save to JSON
output_path = "predicted_answers.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Saved predictions to {output_path}")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating answers: 100%|██████████| 51/51 [03:27<00:00,  4.07s/it]

Saved predictions to predicted_answers.json


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 886.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 869.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


In [ ]:
!pip install transformers peft accelerate


In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


Second adapter training

In [ ]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
model_id = "meta-llama/Llama-3.2-3B"


dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="train")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# First adapter
path_adapter_1 = "/content/"

# Load adapter configuration
config = PeftConfig.from_pretrained(path_adapter_1)

# Load base model with quantization (must match training setup)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False
)


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# Load first adapter and freeze it
model = prepare_model_for_kbit_training(base_model)
model = PeftModel.from_pretrained(base_model, path_adapter_1)
model.print_trainable_parameters()  # should show only LoRA trainable

# # Freeze the weights of this adapter
# for param in model.parameters():
#     param.requires_grad = False

trainable params: 0 || all params: 3,215,043,584 || trainable%: 0.0000


In [ ]:
from peft import prepare_model_for_kbit_training  # if using QLoRA
from peft import LoraConfig

# Define new LoRA config for second adapter
lora_config_v2 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # depends on model architecture
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Add new adapter on top (name it 'lora_v2')
model.add_adapter("lora_v2", lora_config_v2)

# Enable only lora_v2 for training
model.set_adapter("lora_v2")
# for name, param in model.named_parameters():
#     param.requires_grad = "lora_v2" in name


In [ ]:
model.print_trainable_parameters()  # should show only LoRA trainable


trainable params: 1,146,880 || all params: 3,216,190,464 || trainable%: 0.0357


In [ ]:
# Format dataset
def format_example(example):
    prompt = (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer: {example['answer']}"
    )
    return {"text": prompt}

def preprocess(example):
    prompt = f"Context: {example['text']}\nQuestion: {example['question']}\nAnswer:"
    full_text = prompt + " " + example["answer"]

    # Tokenize full text
    tokenized = tokenizer(full_text, truncation=True, padding="max_length", max_length=512)

    # Mask out the prompt portion in the labels so loss is only on the answer
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # Find index where answer starts
    answer_start = len(tokenizer(prompt)["input_ids"])
    labels = [-100] * answer_start + input_ids[answer_start:]

    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(preprocess)

# Tokenize dataset
# MAX_LENGTH = 512
# tokenized_dataset = formatted_dataset.map(
#     lambda e: tokenizer(e["text"], truncation=True, max_length=MAX_LENGTH, padding=False),
#     batched=True,
#     remove_columns=dataset.column_names
# )

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Optimized training arguments
training_args = TrainingArguments(
    output_dir="/content/lora_adapter_v2",
    per_device_train_batch_size=2,          # Increased due to LoRA memory efficiency
    gradient_accumulation_steps=4,           # Adjust based on GPU memory
    num_train_epochs=2,
    learning_rate=3e-4,                     # Lower LR for adapter training
    weight_decay=0.01,
    bf16=True,                              # Use bfloat16
    logging_steps=10,
    optim="paged_adamw_8bit",               # Optimizer for 8-bit models
    save_strategy="no",
    report_to="none",
    gradient_checkpointing=True,
    )




Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset[0].keys()


dict_keys(['text', 'question', 'answer', 'source', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    # tokenizer=tokenizer,
    data_collator=data_collator
    )

# Start training
trainer.train()

# Save adapter weights
model.save_pretrained("lora_adapter_v2")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.833000
20,1.549500
30,1.327700
40,1.400100
50,1.285200
60,1.357700
70,1.364900


In [ ]:
model.eval()
test_dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="test")


In [ ]:
# Format function
def format_prompt(example):
    return (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer:"
    )

In [ ]:

print("\nPrediction for the last test data sample:")
last_sample = test_dataset[-14]
prompt = format_prompt(last_sample)
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

generated = tokenizer.decode(output[0], skip_special_tokens=True)
predicted_answer = generated.split("Answer:")[-1].strip().split("\n")[0]

print(f"context:{last_sample['text']}")
print(f"Question: {last_sample['question']}")
print(f"True Answer: {last_sample['answer']}")
print(f"Predicted Answer: {predicted_answer}")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Prediction for the last test data sample:
context:Bucovina este traversată de râuri importante precum Siretul și Moldava.
Question: Care este un râu important care traversează Bucovina, pe lângă Moldova?
True Answer: Siret
Predicted Answer: Lemne și lemn


In [ ]:
!unzip "/content/lora_v2_3e-4_1 epoch.zip"

Archive:  /content/lora_v2_3e-4_1 epoch.zip
   creating: lora_v2_3e-4_1 epoch/
  inflating: lora_v2_3e-4_1 epoch/adapter_model.safetensors  
  inflating: __MACOSX/lora_v2_3e-4_1 epoch/._adapter_model.safetensors  
  inflating: lora_v2_3e-4_1 epoch/.DS_Store  
  inflating: __MACOSX/lora_v2_3e-4_1 epoch/._.DS_Store  
  inflating: lora_v2_3e-4_1 epoch/README.md  
  inflating: __MACOSX/lora_v2_3e-4_1 epoch/._README.md  
  inflating: lora_v2_3e-4_1 epoch/adapter_config.json  
  inflating: __MACOSX/lora_v2_3e-4_1 epoch/._adapter_config.json  
   creating: lora_v2_3e-4_1 epoch/lora/
  inflating: lora_v2_3e-4_1 epoch/lora/adapter_model.safetensors  
  inflating: __MACOSX/lora_v2_3e-4_1 epoch/lora/._adapter_model.safetensors  
  inflating: lora_v2_3e-4_1 epoch/lora/adapter_config (1).json  
  inflating: __MACOSX/lora_v2_3e-4_1 epoch/lora/._adapter_config (1).json  


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch # Import torch for torch.float16

base_model_name = "meta-llama/Llama-3.2-3B"

# List of paths to your adapters
adapter_paths = [
    "/content/lora_v2_3e-4_1_epoch", # path to adapter 1
    "/content/lora_v2_3e-4_1_epoch/lora", # path to adapter 2
    # Add more adapter paths here if needed
]

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for better performance on supported GPUs
    bnb_4bit_use_double_quant=True
)

# Load base model with appropriate dtype, quantization config, and device mapping
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder="offload" # Optional: for offloading to CPU
)

# Apply adapters sequentially (stacked)
for i, adapter_path in enumerate(adapter_paths):
    if i == 0:
        # Load the first adapter on top of the base model
        model = PeftModel.from_pretrained(model, adapter_path)
    else:
        # Add subsequent adapters on top of the previous one
        model.load_adapter(adapter_path, adapter_name=f"adapter_{i}")

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("Model loaded with stacked adapters and quantization.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded with stacked adapters and quantization.


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import json
from tqdm import tqdm

# # Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# # Load adapter configuration
# adapter_dir = "/content/llama3-qa-lora-adapter"  # Path to saved LoRA adapter
# config = PeftConfig.from_pretrained(adapter_dir)

# # Load base model with quantization (must match training setup)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     quantization_config=bnb_config,
#     device_map="auto",
#     use_cache=False
# )

# # Attach LoRA adapter to base model
# model = PeftModel.from_pretrained(base_model, adapter_dir)
model.eval()

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load test dataset
test_dataset = load_dataset("OnnieNLP/InformationExtractionQA", split="test")

# Format function
def format_prompt(example):
    return (
        "You are a helpful assistant that answers questions based on provided context. Keep answers short.\n\n"
        f"Context: {example['text']}\n\nQuestion: {example['question']}\n\nAnswer:"
    )

# Inference
results = []
for example in tqdm(test_dataset, desc="Generating answers"):
    prompt = format_prompt(example)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.7,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_answer = generated.split("Answer:")[-1].strip()

    results.append({
        "question": example["question"],
        "context": example["text"],
        "true_answer": example["answer"],
        "predicted_answer": predicted_answer
    })

# Save to JSON
output_path = "/content/predicted_answers_2_adapters.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Saved predictions to {output_path}")

Using device: cuda


Generating answers: 100%|██████████| 200/200 [15:47<00:00,  4.74s/it]

Saved predictions to /content/predicted_answers_2_adapters.json
